In [ ]:
# 1. Gerekli Kütüphanelerin Yüklenmesi
!pip install -q -U transformers datasets accelerate peft bitsandbytes trl huggingface_hub
# Install transformers from source
!pip install git+https://github.com/huggingface/transformers.git

# 2. Hugging Face Girişi
from huggingface_hub import notebook_login
notebook_login()

# 3. Model ve Tokenizer Yükleme
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Gemma3TextConfig, TrainingArguments # Import TrainingArguments

model_name = "google/gemma-3-4b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)

# Explicitly use Gemma3TextConfig to initialize the model
config = Gemma3TextConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config, # Pass the config explicitly
    quantization_config=bnb_config,
    device_map="auto",
    token=True
)

# ... (rest of your code remains the same)

# 4. Veri Setini Yükleme ve Formatlama
from datasets import load_dataset

dataset = load_dataset("zinderud/risale")

def format_instruction(sample):
    # Use the existing 'text' key for the causal language model training.
    # You can incorporate 'stringlengths' and 'file_type' if needed.
    return {
        "text": sample['text'] 
    }

dataset = dataset.map(format_instruction)

# 5. Eğitim Konfigürasyonu
from peft import LoraConfig
from trl import SFTTrainer

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=TrainingArguments( # Use TrainingArguments directly
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=500,
        learning_rate=2e-5,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none",
    ),
)

# The 'max_seq_length' should be passed during data preprocessing.
# Example:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=1024)

dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,  # Remove original columns
)

# 6. Eğitimi Başlatma
trainer.train()

# 7. Modeli Kaydetme ve Yükleme
merged_model = model.merge_and_unload()
merged_model.save_pretrained("hugin-risale-gemma-3-4b")
tokenizer.save_pretrained("hugin-risale-gemma-3-4b")

# 8. Hugging Face Hub'a Yükleme
merged_model.push_to_hub("hugin-risale-gemma-3-4b", token=True)
tokenizer.push_to_hub("hugin-risale-gemma-3-4b", token=True)